# Counting Stacks and Queues
### Copyright Luca de Alfaro, 2019-21.  License: CC-BY-NC.

## Stacks, Queues, and Their Counting Versions

A stack is a data structure with two operations: push, and pop.  Picture it as a pile of dishes sitting on a counter.  A push operation places a dish on top of the pile.  A pop operation returns the dish on top of the pile, or None if the pile is empty, that is, contains no dishes.  A "dish" can be any Python object.

A queue is a data structure with two operations: put, and get.  Imagine it as a stack of books horizontally on a shelf.  A put operation adds the book to the left end of the books on the shelf; a get operation gets the book from the right end of the shelf.  

Thus, the difference between a stack and a queue is that the stack is FILO (First In, Last Out), whereas the queue is FIFO (First In, First Out).  Elements in a stack are retrieved newest first.
Elements in a queue are retrieved in the order they were put in, oldest first.

We will implement here these data structures, with a small twist: we will also introduce _counting_ versions of them, which avoid keeping multiple identical copies of objects in a row.



Let us begin by implementing a plain vanilla stack.

In [1]:
class Stack(object):

    def __init__(self):
        self.stack = []

    def __repr__(self):
        """Defining a __repr__ function will enable us to print the
        stack contents, and facilitate debugging."""
        return repr(self.stack) # Good enough.

    def push(self, x):
        """The "top" of the stack is the end of the list."""
        self.stack.append(x)

    def pop(self):
        return self.stack.pop() if len(self.stack) > 0 else None

    def isempty(self):
        return len(self.stack) == 0


Let's see how this works.

In [2]:
s = Stack()
print(s.pop())
s.push('a')
s.push('b')
print(s.pop())
print(s.pop())
print(s.pop())

None
b
a
None


Ok!  The definition of a queue is similar.

In [3]:
class Queue(object):

    def __init__(self):
        self.queue = []

    def __repr__(self):
        """Defining a __repr__ function will enable us to print the
        queue contents, and facilitate debugging."""
        return repr(self.queue) # Good enough.

    def add(self, x):
        self.queue.append(x)

    def get(self):
        # This is the only difference compared to the stack above.
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0


Let's see how it works.

In [4]:
s = Queue()
print(s.get())
s.add('a')
s.add('b')
print(s.get())
print(s.get())
print(s.get())

None
a
b
None


As you see, in a queue, the elements are retrieved in the same order in which they were added.

Python experts might note that, for a queue, we would do better by using the [`collections.deque` class](https://docs.python.org/3.7/library/collections.html#collections.deque), rather than the list class, to make the `pop(0)` operation more efficient; in lists, it takes time proportional to the length of the list; in deques, it takes constant time.  For small lists, however, the difference is negligible.


We now consider a use case in which we may need to put in the queue or stack many repeated copies of the same object.  For instance, assume that the queue is used to store events, and assume that some event may end up being repeated many times in a row.  As an example, the events can be "s", for the tick of a second, "m", when the minute advances, and "h", when the hour advances.  There will be 60 consecutive "s" events between any two "m" events, and it seems a waste to store so many consecutive identical events.  Storing many identical things in a row is akin to counting in unary notation, after all.  We would be better off storing the repeated elements only once, along with a count of the number of times they occur.  Let's develop a queue using this idea (a stack can be done similarly).

To facilitate debugging, we will implement counting queues in two fashions: first in a silly fashion, implementing their correct interface, but without implementing the smart way of storing elements with their count, and then later in the proper fashion.
Implementing things the silly way is often useful.  For one thing, it's easier, which other things being equal is an advantage.  For another, it will let you postpone the difficult implementation, so that you can do it once you really have enough data to support the belief that repeated elements will be common.  Finally, simple  implementations are useful in testing, as you can compare the behavior of more complex and efficient implementations with that of simpler, if inefficient, ones.  You can even profile your code later, to decide whether the complication of adopting the more refined implementation was worth it.

In [5]:
class NotQuiteCountingQueue(object):

    def __init__(self):
        self.queue = []

    def __repr__(self):
        """Defining a __repr__ function will enable us to print the
        queue contents, and facilitate debugging."""
        return repr(self.queue) # Good enough.

    def add(self, x, count=1):
        """When we push an element, we can push it with an optional count."""
        # This is a devilish trick, but if you multiply a list, you get multiple
        # copies of it concatenated.
        self.queue = self.queue + [x] * count

    def get(self):
        return self.queue.pop(0) if len(self.queue) > 0 else None

    def isempty(self):
        return len(self.queue) == 0

    def length(self):
        return len(self.queue)


Let us see how this works.

In [6]:
q = NotQuiteCountingQueue()
q.add('a')
q.add('b', count=5)
q.add('c', count=2)
while not q.isempty():
    print(q.get())

a
b
b
b
b
b
c
c


Let's write our smart implementation now.  In the queue, we will store pairs $(x, n)$, where $x$ is an element and $n$ is the count of the number of occurrences of $x$.

In [7]:
class CountingQueue(object):

    def __init__(self):
        self.queue = []

    def __repr__(self):
        return repr(self.queue)

    def add(self, x, count=1):
        # If the element is the same as the last element, we simply
        # increment the count.  This assumes we can test equality of
        # elements.
        if len(self.queue) > 0:
            xx, cc = self.queue[-1]
            if xx == x:
                self.queue[-1] = (xx, cc + count)
            else:
                self.queue.append((x, count))
        else:
            self.queue = [(x, count)]

    def get(self):
        if len(self.queue) == 0:
            return None
        x, c = self.queue[0]
        if c == 1:
            self.queue.pop(0)
            return x
        else:
            self.queue[0] = (x, c - 1)
            return x

    def isempty(self):
        # Since the count of an element is never 0, we can just check
        # whether the queue is empty.
        return len(self.queue) == 0


Let's put this to the same test as before, printing the queue contents at each step to see what is going on.

In [8]:
q = CountingQueue()
q.add('a')
print(q)
q.add('b', count=5)
print(q)
q.add('c', count=2)
print(q)
while not q.isempty():
    print(q.get())
    print(q)

[('a', 1)]
[('a', 1), ('b', 5)]
[('a', 1), ('b', 5), ('c', 2)]
a
[('b', 5), ('c', 2)]
b
[('b', 4), ('c', 2)]
b
[('b', 3), ('c', 2)]
b
[('b', 2), ('c', 2)]
b
[('b', 1), ('c', 2)]
b
[('c', 2)]
c
[('c', 1)]
c
[]


It works!  And notice that it works even if we add elements one by one.

In [9]:
q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
print(q)

[('a', 10), ('b', 1), ('c', 6)]


In [10]:
#@title Testing helper

def check_equal(x, y, msg=None):
    if x == y:
        if msg is not None:
            print(msg, ": Success")
    else:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)

## The `__len__` Method

If you want to take the length of an object, as in

    len(someobject)

then `someobject` must have a `__len__` method, which should return the length.
Here is a wrong implementation, which always returns length 2.  The implementation is wrong, but it shows how `__len__` should be implemented.

In [11]:
# We define a function...
def wronglength(self):
    return 2

#... and we assign it to the __len__ methods.
CountingQueue.__len__ = wronglength

What we did before is a bit un-orthodox.  We should have really added the definition of `__len__` into the declaration of the class, like this:

    class CountingQueue(object):

        def __init__(self):
            self.queue = []

        # ... etc etc...

        def __len__(self):
            return 2

In these class notebooks, however, to avoid redefining classes from scratch all the time, we will often use the trick of defining a function, and then assigning it to a class method.

**Exercise:** Define a correct function `__len__`, which returns the number of elements in a counting queue.

In [12]:
#@title Exercise: implement `__len__` for a counting queue

def countingqueue_len(self):
    """Returns the number of elements in the queue."""
    ### YOUR SOLUTION HERE
    return sum(count for _, count in self.queue)
# This is a way to add a method to a class once the class
# has already been defined.
CountingQueue.__len__ = countingqueue_len

In [13]:
# Tests 10 points: `__len__`

q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
check_equal(len(q), 17)


**Exercise:** Implement counting stacks.

## The `__iter__` Method

We would like to be able to have a way of iterating over elements in our counting queue.  
This can be used, for instance, to print them, or to process the elements in some way.

Precisely, we would like to have a way of writing, for a counting queue `q`:

    for el in q:
        print el

and we would like this to print all queue elements, in order.

The way to achieve this is to define an `__iter__` method that acts as a generator for the elements.

Doing this for a normal (non-counting) queue is easy.  Note how we are again using our hacky syntax for adding a method post-definition to a class.

In [14]:
def queue_iter_elements(self):
    for el in self.queue:
        yield el

Queue.__iter__ = queue_iter_elements

In [15]:
normal_queue = Queue()
normal_queue.add('a')
normal_queue.add('b')
normal_queue.add('c')

for el in normal_queue:
    print(el)

a
b
c


Note that we cannot use our `queue_iter_elements` on a counting queue; we would get a wrong result:

In [16]:
CountingQueue.__iter__ = queue_iter_elements

q = CountingQueue()
for i in range(5):
    q.add(i)
    q.add(i)
for el in q:
    print(el)

(0, 2)
(1, 2)
(2, 2)
(3, 2)
(4, 2)


**Exercise:** Write an iterator for counting queue that correctly iterates over the element in the counting queue.

In [17]:
#@title Exercise: Write an iterator for CountingQueue

# Note: it can be done elegantly in 3 lines of code.

def countingqueue_iter_elements(self):
    """Iterates through all the elements of the queue,
    without removing them."""
    ### YOUR SOLUTION HERE
    for element, count in self.queue:
        for _ in range(count):
            yield element

CountingQueue.__iter__ = countingqueue_iter_elements

In [18]:
# Tests 10 points: for `CountingQueue.__iter__`

q = CountingQueue()
for i in range(10):
    q.add('a')
q.add('b')
for i in range(3):
    q.add('c', count=2)
l1 = [x for x in q]
l2 = []
while not q.isempty():
    l2.append(q.get())
check_equal(l1, l2)
